In [16]:
import pandas as pd 
from datetime import datetime

In [3]:
df = pd.read_csv('../data/season_data.csv')
df

,Unnamed: 0,RaceNumber,Date,Track,Cars,Winner,WinnerStart,WinnerMake,TrackLength,TrackSurface,MilesRun,Purse,PoleSpeed,NumCautions,LapsUnderCaution,AvgSpeed,LeadChanges,Year
0,0,1,02/20/00,Daytona,43,Dale Jarrett,1,Ford,2.500,P,500,"7,646,975",191.091,6,24,155.669,9,2000
1,1,2,02/27/00,Rockingham,43,Bobby Labonte,3,Pontiac,1.017,P,400,"1,744,304",158.035,4,22,127.875,22,2000
2,2,3,03/05/00,Las Vegas,43,Jeff Burton,11,Ford,1.500,P,222,"3,572,500",172.563,2,29,119.982,13,2000
3,3,4,03/12/00,Atlanta,43,Dale Earnhardt,35,Chevrolet,1.540,P,501,"1,792,435",192.574,10,62,131.759,30,2000
4,4,5,03/19/00,Darlington,43,Ward Burton,3,Pontiac,1.366,P,400,"1,792,036",172.662,5,30,128.076,13,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,18,19,07/23/20,Kansas,40,Denny Hamlin,10,Toyota,1.500,P,401,,DFP,11,47,121.835,21,2020
737,19,20,08/02/20,Loudon,38,Brad Keselowski,4,Ford,1.058,P,318,,DFP,11,52,100.372,22,2020
738,20,21,08/08/20,Michigan,39,Kevin Harvick,3,Ford,2.000,P,322,,DFP,9,43,124.712,12,2020
739,21,22,08/09/20,Michigan,39,Kevin Harvick,20,Ford,2.000,P,312,,NTT,5,24,144.463,10,2020


In [4]:
df.dtypes

Unnamed: 0            int64
RaceNumber            int64
Date                 object
Track                object
Cars                  int64
Winner               object
WinnerStart           int64
WinnerMake           object
TrackLength         float64
TrackSurface         object
MilesRun              int64
Purse                object
PoleSpeed            object
NumCautions           int64
LapsUnderCaution      int64
AvgSpeed            float64
LeadChanges           int64
Year                  int64
dtype: object

In [5]:
df['PoleSpeed'] = pd.to_numeric(df['PoleSpeed'], errors='coerce')
df['PoleSpeed']

0      191.091
1      158.035
2      172.563
3      192.574
4      172.662
        ...   
736        NaN
737        NaN
738        NaN
739        NaN
740        NaN
Name: PoleSpeed, Length: 741, dtype: float64

In [13]:
df['Purse'] = pd.to_numeric(df['Purse'].apply(lambda x: x.replace(',', '')), errors='coerce')
df['Purse']

0      7646975.0
1      1744304.0
2      3572500.0
3      1792435.0
4      1792036.0
         ...    
736          NaN
737          NaN
738          NaN
739          NaN
740          NaN
Name: Purse, Length: 741, dtype: float64

In [15]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')

In [18]:
def stage_adjust_cautions(row):
    stage_start = datetime(2017, 1, 1)
    base_cautions = row['NumCautions']
    race_date = row['Date']

    # stage cautions introduced for 2017 season, everything before that doesn't need adjustment
    if race_date < stage_start:
        return base_cautions
    
    # Coke 600 at Charlotte has 4 stages for some reason
    # potentially incorrect if race is rain-shortened
    if row['Track'] == 'Charlotte' and row['MilesRun'] > 500:
        return base_cautions - 3

    # everything else has 3 stages (again, unless rain-shortened)
    return base_cautions - 2

In [24]:
df['StageAdjustCautions'] = df.apply(stage_adjust_cautions, axis=1)
df[['Date', 'Track', 'NumCautions','StageAdjustCautions']]

,Date,Track,NumCautions,StageAdjustCautions
0,2000-02-20,Daytona,6,6
1,2000-02-27,Rockingham,4,4
2,2000-03-05,Las Vegas,2,2
3,2000-03-12,Atlanta,10,10
4,2000-03-19,Darlington,5,5
...,...,...,...,...
736,2020-07-23,Kansas,11,9
737,2020-08-02,Loudon,11,9
738,2020-08-08,Michigan,9,7
739,2020-08-09,Michigan,5,3
